# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 5 2022 8:30AM,243505,10,BPS20004,Beach Products Inc,Released
1,Aug 5 2022 8:30AM,243502,10,SNAP30902376,"Snap Medical Industries, LLC",Released
2,Aug 5 2022 8:30AM,243502,10,SNAP30968574,"Snap Medical Industries, LLC",Released
3,Aug 5 2022 8:30AM,243502,10,SNAP59919,"Snap Medical Industries, LLC",Released
4,Aug 4 2022 3:00PM,243482,10,0085924420,ISDIN Corporation,Executing
5,Aug 4 2022 3:00PM,243482,10,0085924432,ISDIN Corporation,Executing
6,Aug 4 2022 3:00PM,243482,10,0085924433,ISDIN Corporation,Executing
7,Aug 4 2022 3:00PM,243482,10,0085924430,ISDIN Corporation,Executing
8,Aug 4 2022 3:00PM,243482,10,0085924434,ISDIN Corporation,Executing
9,Aug 4 2022 3:00PM,243482,10,0085924473,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,243484,Released,2
14,243485,Released,7
15,243497,Executing,1
16,243502,Released,3
17,243505,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243484,NaN,2.0
243485,NaN,7.0
243497,1.0,NaN
243502,NaN,3.0
243505,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243451,1.0,4.0
243455,0.0,1.0
243457,17.0,1.0
243461,3.0,16.0
243464,0.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243451,1,4
243455,0,1
243457,17,1
243461,3,16
243464,0,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243451,1,4
1,243455,0,1
2,243457,17,1
3,243461,3,16
4,243464,0,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243451,1,4
1,243455,,1
2,243457,17,1
3,243461,3,16
4,243464,,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 5 2022 8:30AM,243505,10,Beach Products Inc
1,Aug 5 2022 8:30AM,243502,10,"Snap Medical Industries, LLC"
4,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation
27,Aug 4 2022 2:58PM,243497,12,Hush Hush
28,Aug 4 2022 2:19PM,243485,20,Alumier Labs Inc.
35,Aug 4 2022 2:19PM,243484,20,Alumier Labs Inc.
37,Aug 4 2022 2:14PM,243483,21,"NBTY Global, Inc."
38,Aug 4 2022 1:44PM,243481,10,ISDIN Corporation
68,Aug 4 2022 1:24PM,243478,10,"Citieffe, Inc."
69,Aug 4 2022 12:40PM,243464,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 5 2022 8:30AM,243505,10,Beach Products Inc,,1
1,Aug 5 2022 8:30AM,243502,10,"Snap Medical Industries, LLC",,3
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,6,17
3,Aug 4 2022 2:58PM,243497,12,Hush Hush,1,
4,Aug 4 2022 2:19PM,243485,20,Alumier Labs Inc.,,7
5,Aug 4 2022 2:19PM,243484,20,Alumier Labs Inc.,,2
6,Aug 4 2022 2:14PM,243483,21,"NBTY Global, Inc.",,1
7,Aug 4 2022 1:44PM,243481,10,ISDIN Corporation,,30
8,Aug 4 2022 1:24PM,243478,10,"Citieffe, Inc.",,1
9,Aug 4 2022 12:40PM,243464,10,ISDIN Corporation,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 8:30AM,243505,10,Beach Products Inc,1,
1,Aug 5 2022 8:30AM,243502,10,"Snap Medical Industries, LLC",3,
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,17,6
3,Aug 4 2022 2:58PM,243497,12,Hush Hush,,1
4,Aug 4 2022 2:19PM,243485,20,Alumier Labs Inc.,7,
5,Aug 4 2022 2:19PM,243484,20,Alumier Labs Inc.,2,
6,Aug 4 2022 2:14PM,243483,21,"NBTY Global, Inc.",1,
7,Aug 4 2022 1:44PM,243481,10,ISDIN Corporation,30,
8,Aug 4 2022 1:24PM,243478,10,"Citieffe, Inc.",1,
9,Aug 4 2022 12:40PM,243464,10,ISDIN Corporation,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 8:30AM,243505,10,Beach Products Inc,1,
1,Aug 5 2022 8:30AM,243502,10,"Snap Medical Industries, LLC",3,
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,17,6
3,Aug 4 2022 2:58PM,243497,12,Hush Hush,,1
4,Aug 4 2022 2:19PM,243485,20,Alumier Labs Inc.,7,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 8:30AM,243505,10,Beach Products Inc,1.0,NaN
1,Aug 5 2022 8:30AM,243502,10,"Snap Medical Industries, LLC",3.0,NaN
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,17.0,6.0
3,Aug 4 2022 2:58PM,243497,12,Hush Hush,NaN,1.0
4,Aug 4 2022 2:19PM,243485,20,Alumier Labs Inc.,7.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 5 2022 8:30AM,243505,10,Beach Products Inc,1.0,0.0
1,Aug 5 2022 8:30AM,243502,10,"Snap Medical Industries, LLC",3.0,0.0
2,Aug 4 2022 3:00PM,243482,10,ISDIN Corporation,17.0,6.0
3,Aug 4 2022 2:58PM,243497,12,Hush Hush,0.0,1.0
4,Aug 4 2022 2:19PM,243485,20,Alumier Labs Inc.,7.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1704367,66.0,6.0
12,243497,0.0,1.0
19,243451,4.0,1.0
20,973887,26.0,20.0
21,243483,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1704367,66.0,6.0
1,12,243497,0.0,1.0
2,19,243451,4.0,1.0
3,20,973887,26.0,20.0
4,21,243483,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,66.0,6.0
1,12,0.0,1.0
2,19,4.0,1.0
3,20,26.0,20.0
4,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,66.0
1,12,Released,0.0
2,19,Released,4.0
3,20,Released,26.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,21
Status,,,,,
Executing,6.0,1.0,1.0,20.0,0.0
Released,66.0,0.0,4.0,26.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,21
0,Executing,6.0,1.0,1.0,20.0,0.0
1,Released,66.0,0.0,4.0,26.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,21
0,Executing,6.0,1.0,1.0,20.0,0.0
1,Released,66.0,0.0,4.0,26.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()